## Question 7

## Logistic Regression Vs Bayesian Classifier

In [1]:
#reading data sets
train_data <- read.csv("Task1E_train.csv")
test_data <- read.csv("Task1E_test.csv")

In [2]:
head(train_data)

x1,x2,y
<dbl>,<dbl>,<int>
-0.4367052,3.171451,-1
5.1048972,1.227815,1
3.9974518,1.049502,1
2.5354540,1.242902,1
4.2624556,2.551909,1
1.5505866,2.595164,-1


In [3]:
dim(train_data)

[1] 500   3

In [4]:
head(test_data)

x1,x2,y
<dbl>,<dbl>,<int>
3.4781571,-0.51546992,1
4.8959781,0.95320554,1
-0.5783557,5.44553527,-1
3.8417831,0.65759443,1
2.2649154,4.18082840,-1
3.8550566,-0.08328894,1


In [5]:
#splitting the train and testing data points
train.datas <- train_data[,-3]
train.labels <- train_data[,3]
test.datas <- test_data[,-3]
test.labels <- test_data[,3]

In [6]:
# auxiliary function that predicts class labels
predict <- function(w, X, c0, c1){
    sig <- sigmoid(w, X)
    return(ifelse(sig>0.5, c1,c0))
}
    
# auxiliary function that calculate a cost function
cost <- function (w, X, T, c0){
    sig <- sigmoid(w, X)
    return(sum(ifelse(T==c0, 1-sig, sig)))
}

In [7]:
# Sigmoid function (=p(C1|X))
sigmoid <- function(w, x){
    return(1.0/(1.0+exp(-w%*%t(cbind(1,x)))))    
}

In [8]:
# Initializations
tau.max <- 1000 # maximum number of iterations
eta <- 0.01 # learning rate
epsilon <- 0.01 # a threshold on the cost (to terminate the process)
tau <- 1 # iteration counter
terminate <- FALSE

In [9]:
logisticregression <- function(traindata,trainlabel,trainlen){
    trainlen = nrow(traindata)
    
    X <- as.matrix(traindata) 
    T <- ifelse(trainlabel==1,0,1) 
    
    W <- matrix(,nrow=tau.max, ncol=(ncol(X)+1)) 
    W[1,] <- runif(ncol(W))
    
    Y <- sigmoid(W[1,],X) #calling sigmoid function
    
    costs <- data.frame('tau'=1:tau.max) 
    costs[1, 'cost'] <- cost(W[1,],X,T, 1)
    
    # check termination criteria
    while(!terminate){
        
        terminate <- tau >= tau.max | cost(W[tau,],X,T, 1)<=epsilon
        # shuffle data
        train.index <- sample(1:trainlen, trainlen, replace = FALSE)
    
        X <- X[train.index,]
        T <- T[train.index]
        
         # for each sample datapoint
        for (i in 1:trainlen){
            
            #check criteria
            if (tau >= tau.max | cost(W[tau,],X,T, 1) <=epsilon) {terminate<-TRUE;break}
            
            Y <- sigmoid(W[tau,],X)
            
            # Update the weights
            W[(tau+1),] <- W[tau,] - eta * (Y[i]-T[i]) * cbind(1, t(X[i,]))
            
            costs[(tau+1), 'cost'] <- cost(W[tau,],X,T, 1) #update the costs
            
            tau <- tau + 1 #increase tau value
            
            eta = eta * 0.999 #increase learning rate
    }
}

    costs <- costs[1:tau, ] 

    w <- W[tau,]
    
    return (w) #finally return weighing vector
    
}
    

### Using the first 5 data points from the training set, train a LR model,and compute their training and testing errors.

In [28]:
train_dataa_points <- train.datas[1:5,]
train_labell_points <- train.labels[1:5]
train_lengthh_points <- 5
weighing_vector_valuee <- logisticregression(train_dataa_points, train_labell_points, train_lengthh_points)
errors_train <- sum(train_labell_points != predict(weighing_vector_valuee, train_dataa_points, 1, -1))/nrow(train_dataa_points)
cat("The training error for the first 5 data points of logistic regression is ")
cat(errors_train)
cat("\n")
errors_test <- sum(test.labels != predict(weighing_vector_valuee, test.datas, 1, -1))/nrow(test.datas)
cat("The testing error for the first 5 data points of logistic regression is ")
cat(errors_test)

The training error for the first 5 data points of logistic regression is 0
The testing error for the first 5 data points of logistic regression is 0.124

## Question

In a “for loop”, increase the
size of training set (5 data points at a time), retrain the models and
calculate their training and testing errors until all training data points 

In [29]:
logistic_regression_predictions <- data.frame('size' = rep(0,100), 'testing_error' = rep(0,100), 'training_error' = rep(0,100))
length_train <- seq(5, nrow(train.datas),by = 5)
for (i in 1:length(length_train)){    
    
    train_dataa <- train.datas[1:length_train[i],]
    train_labell <- train.labels[1:length_train[i]]
    train_lengthh <- length_train[i]
    weighing_vector_value <- logisticregression(train_dataa, train_labell, train_lengthh)
    errors_test <- sum(test.labels != predict(weighing_vector_value, test.datas, 1, -1))/nrow(test.datas)
    errors_train <- sum(train_labell != predict(weighing_vector_value, train_dataa, 1, -1))/nrow(train_dataa)
    
    logistic_regression_predictions[i, 'size'] <- train_lengthh
    logistic_regression_predictions[i, 'testing_error'] <- errors_test
    logistic_regression_predictions[i, 'training_error'] <- errors_train

}

In [30]:
head(logistic_regression_predictions)

size,testing_error,training_error
<dbl>,<dbl>,<dbl>
5,0.118,0.00000000
10,0.020,0.00000000
15,0.014,0.00000000
20,0.016,0.05000000
25,0.014,0.04000000
30,0.014,0.03333333


In [31]:
c0 <- 1
c1 <- -1

### Using the first 5 data points from the training set, train a BC model,and compute their training and testing errors.

In [32]:
train_dataa_bc <- train.datas[1:5,]
train_label_bc <- train.labels[1:5]
train_length_bc <- 5
 
# Class probabilities
p0.hat <- sum(train_label_bc==c0)/nrow(train_dataa_bc)
p1.hat <- sum(train_label_bc==c1)/nrow(train_dataa_bc)

# Class means
mu0.hat <- colMeans(train_dataa_bc[train_label_bc==c0,])
mu1.hat <- colMeans(train_dataa_bc[train_label_bc==c1,])

# class covariance matrices
sigma0.hat <- var(train_dataa_bc[train_label_bc==c0,])
sigma1.hat <- var(train_dataa_bc[train_label_bc==c1,])

# shared covariance matrix
sigma.hat <- p0.hat * sigma0.hat + p1.hat * sigma1.hat 

# calculate posteriors
posterior0 <- p0.hat*dmvnorm(x=train_dataa_bc, mean=mu0.hat, sigma=sigma.hat)
posterior1 <- p1.hat*dmvnorm(x=train_dataa_bc, mean=mu1.hat, sigma=sigma.hat)

# calculate predictions
train.predict <- ifelse(posterior0 > posterior1, c0, c1)
test.predict <- ifelse(p0.hat*dmvnorm(x=test.datas, mean=mu0.hat, sigma=sigma.hat) > p1.hat*dmvnorm(x=test.datas, mean=mu1.hat, sigma=sigma.hat), c0, c1)
 
#finding errors for training and testing
errors_test1 <- sum(test.predict != test.labels)/nrow(test.datas)
errors_train1 <- sum(train.predict != train_label_bc)/nrow(train_dataa_bc)

In [33]:
cat("The training error for the first 5 data points of bayesian classifier is ")
cat(errors_train1)
cat("\n")
cat("The testing error for the first 5 data points of bayesian regression is ")
cat(errors_test1)

The training error for the first 5 data points of bayesian classifier is 0.8
The testing error for the first 5 data points of bayesian regression is 0.684

## Question

In a “for loop”, increase the
size of training set (5 data points at a time), retrain the models and
calculate their training and testing errors until all training data points 

In [34]:
library(mvtnorm)

bayesian_classifier_predictions <- data.frame('size' = rep(0,100), 'testing_error' = rep(0,100), 'training_error' = rep(0,100))

length_train <- seq(5, nrow(train.datas),by = 5)

for (i in 1:length(length_train)){    
    
    #getting the required slices of training data and its labels
    train_dataa_bc <- train.datas[1:length_train[i],]
    train_label_bc <- train.labels[1:length_train[i]]
    train_length_bc <- length_train[i]
    
    # Class probabilities
    p0.hat <- sum(train_label_bc==c0)/nrow(train_dataa_bc)
    p1.hat <- sum(train_label_bc==c1)/nrow(train_dataa_bc)

    # Class means
    mu0.hat <- colMeans(train_dataa_bc[train_label_bc==c0,])
    mu1.hat <- colMeans(train_dataa_bc[train_label_bc==c1,])

    # class covariance matrices
    sigma0.hat <- var(train_dataa_bc[train_label_bc==c0,])
    sigma1.hat <- var(train_dataa_bc[train_label_bc==c1,])

    # shared covariance matrix
    sigma.hat <- p0.hat * sigma0.hat + p1.hat * sigma1.hat 

    # calculate posteriors
    posterior0 <- p0.hat*dmvnorm(x=train_dataa_bc, mean=mu0.hat, sigma=sigma.hat)
    posterior1 <- p1.hat*dmvnorm(x=train_dataa_bc, mean=mu1.hat, sigma=sigma.hat)

    # calculate predictions
    train.predict <- ifelse(posterior0 > posterior1, c0, c1)
    test.predict <- ifelse(p0.hat*dmvnorm(x=test.datas, mean=mu0.hat, sigma=sigma.hat) > p1.hat*dmvnorm(x=test.datas, mean=mu1.hat, sigma=sigma.hat), c0, c1)
    
    #finding errors for training and testing
    errors_test <- sum(test.predict != test.labels)/nrow(test.datas)*100
    errors_train <- sum(train.predict != train_label_bc)/nrow(train_dataa_bc)*100
    
    #placing the errors in a data frame
    bayesian_classifier_predictions[i , 'size'] <- train_length_bc
    bayesian_classifier_predictions[i, 'testing_error'] <- errors_test
    bayesian_classifier_predictions[i, 'training_error'] <- errors_train

}

In [35]:
head(bayesian_classifier_predictions)

size,testing_error,training_error
<dbl>,<dbl>,<dbl>
5,68.4,80.000000
10,1.4,0.000000
15,1.8,0.000000
20,1.8,5.000000
25,2.4,4.000000
30,1.6,6.666667


### Visualizing the plots

In [36]:
options(repr.plot.width = 10, repr.plot.height = 7)
plot(logistic_regression_predictions$size, logistic_regression_predictions$training_error, type = "l", col = "red",xlab = "Size", ylab = "Error" , lwd = 3)
title("LR vs BC training error")
par(new=TRUE)
plot(bayesian_classifier_predictions$size, bayesian_classifier_predictions$training_error, type = "l", col = "green",xlab = "Size", ylab = "Error", lwd = 3)
legend(x="topright", legend=c("Logistic Training","Bayesian Training"),
       fill=c("red","green"))

In [38]:
options(repr.plot.width = 10, repr.plot.height = 7)
plot(logistic_regression_predictions$size, logistic_regression_predictions$testing_error, type = "l", col = "red",xlab = "Size", ylab = "Error", lwd = 3)
title("LR vs BC testing error")
par(new=TRUE)
plot(bayesian_classifier_predictions$size, bayesian_classifier_predictions$testing_error, type = "l", col = "green",xlab = "Size", ylab = "Error", lwd = 3)
legend(x="topright", legend=c("Logistic Testing","Bayesian Testing"),
       fill=c("red","green"))

## Question 7.3

### a. What does happen for each classifier when the number of training data points is increased?

1.Training Errors

BC Classifier:- When number of training data points are increasing, the training error rate is decreasing and then remaining almost constant, for further increase.

LR Classifier:- When number of training data points are increasing, we can observe the greater fluctuations, at some sizes of data sets , it is increasing and at some sizes of data sets it is decreasing. 

But, overall we can see the decrease in the training error rate with increase in training data set.

Therefore, we can conclude that with increase of training data points, the training error rate decreases, because of data being sampled with large data points.

2.Testing Errors

BC Classifier:- When number of training data points are increasing, the testing error rate is decreasing and then remaining almost constant, for further increase.

LR Classifier:- When number of training data points are increasing, the testing error rate is decreasing , with a little bit of fluctuations.

Therefore, we can conclude that with increase of training data points, the testing error rate decreases, because of data being sampled with large data points, and can be more generalized to a new unseen data.

Compared to training errors, testing errors have come to zero fastly, because with increase of data points in training data, it can learn a training data well and can respond well to the new unseen data.

### b. Which classifier is best suited when the training set is small, and which is best suited when the training set is big?

From the obtained graphs above, Logistic Regression is suited best if the training set is small, because it has observed a lower error value at a smaller sample size of data. 

When the data points are increasing, Bayesian Classifier, is best suited because the overall error rate observed for Bayesian is less when comapared to Logistic, where logistic is having some fluctuations.


### c. Justify your observations in previous questions (III.a & III.b) by providing some speculations and possible reasons.

1.In general Bayesian works better only if it has less features. As number of features are less in the given data set, Bayesian model works better for the larger data set.

2.Compared to training errors, testing errors have come to zero fastly, because with increase of data points in training data, it can learn a training data well and can respond well to the new unseen data.
